# Find duplicate candidates


Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and is used to find potential duplicated catchments within the dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* shapely
* textdistance
* tqdm
* warnings

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* results/estreams_gauging_stations.csv

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References


## Observations
* As this step is rather qualitative, we believe that the users can also adapt the conditons accordinly. 


# Import modules

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import tqdm as tqdm
import os
import warnings
import textdistance
from shapely.geometry import Point

# Configurations

In [2]:
# Only editable variable:
# Relative path to your local directory
PATH = ".."
# Suppress all warnings
warnings.filterwarnings("ignore")

# Constrains
JARO_THRESHOLD = 0.6
SPATIAL_THRESHOLD = 1000
PROVIDER_THRESHOLD = 0.9
SPATIAL_PROVIDER_THRESHOLD = 50
AREA_THRESHOLD = 0.01

* #### The users should NOT change anything in the code below here.


In [3]:
# Non-editable variables:
PATH_OUTPUT = "results/"
# Set the directory:
os.chdir(PATH)

# Import data
## Streamflow gauges network

In [4]:
network_estreams = pd.read_csv('results/estreams_gauging_stations.csv', encoding='utf-8')
network_estreams.set_index("basin_id", inplace = True)
network_estreams

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,area,...,area_calc,area_flag,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days
basin_id,,,,,,,,,,,,,,,,,,,,,
AT000001,200014,Bangs,AT,AT_EHYD,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,...,4668.379,0,-0.440608,1996-01-01,2021-12-31,26,312,9497,0.0,9497
AT000002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,102.0,...,102.287,0,-0.281373,1958-10-01,2021-12-31,64,759,23103,0.0,23103
AT000003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,535.2,...,536.299,0,-0.205344,1985-01-02,2021-12-31,37,444,13513,0.0,13513
AT000004,200592,Kloesterle (OEBB),AT,AT_EHYD,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,...,66.286,0,0.471471,1998-01-02,2021-12-31,24,288,8765,0.0,8765
AT000005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,...,72.448,0,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR0017,6682300,BASHTANOVKA,UA,UA_GRDC,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,...,325.370,0,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652
UAGR0018,6682500,YALTA,UA,UA_GRDC,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,...,47.594,0,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652
UAGR0019,6683010,PIONERSKOE,UA,UA_GRDC,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,...,244.731,1,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652


# Processing
- Pre-process the dataset

In [8]:
# If we want to clip the data to be used:
df = network_estreams.iloc[0:1000, :]

# Create a GeoDataFrame from DataFrame with WGS 84 coordinates
geometry = [Point(lon, lat) for lon, lat in zip(network_estreams['lon'], network_estreams['lat'])]
gdf_wgs84 = gpd.GeoDataFrame(network_estreams, geometry=geometry, crs='EPSG:4326')

# Reproject the GeoDataFrame to ETRS89 LAEA (EPSG:3035)
gdf_etrs89 = gdf_wgs84.to_crs(epsg=3035)

df = gdf_etrs89.loc[df.index, :]

- Compute the distances

In [9]:
# Create a dictionary to store distances
distances = {}

# Calculate Jaro-Winkler distance for each unique pair of 'gauge_name'
for i, row1 in tqdm.tqdm(df.iterrows()):
    for j, row2 in gdf_etrs89.iterrows():
        # Skip self-comparisons
        if i != j:
            # Calculate gauge name distance
            try:
                gauge_distance = textdistance.jaro_winkler(row1['gauge_name'].lower(), row2['gauge_name'].lower())
            except:
                gauge_distance = np.nan
                
            # Calculate river distance
            try:
                river_distance = textdistance.jaro_winkler(row1['river'].lower(), row2['river'].lower())
            except:
                river_distance = np.nan
                
            provider_distance = row1['gauge_provider'].lower() == row2['gauge_provider'].lower()          
        
            # Calculate distance between points
            point1 = row1['geometry']
            point2 = row2['geometry']
            point_distance = point1.distance(point2)
            
            # Calculate area_calc difference
            area_calc_diff = abs(row1['area_calc'] - row2['area_calc']) / max(row1['area_calc'], row2['area_calc'])
            
            # Store distances along with first and second gauge indices only if gauge_distance > 0.9
            if (gauge_distance > JARO_THRESHOLD) & (river_distance > JARO_THRESHOLD) & (point_distance < SPATIAL_THRESHOLD) & (provider_distance == False):
                distances[(row1['gauge_name'], row2['gauge_name'])] = {
                    'gauge_first_index': i, 
                    'gauge_second_index': j,
                    'gauge_distance': gauge_distance, 
                    'river_distance': river_distance,
                    'point_distance': point_distance,
                    'provider_distance': provider_distance
                }
                
            # Additional condition: if provider_distance is True and point_distance < 250m and area_calc_diff <= 0.01
            if provider_distance and point_distance < SPATIAL_PROVIDER_THRESHOLD and area_calc_diff <= AREA_THRESHOLD:
                distances[(row1['gauge_name'], row2['gauge_name'])] = {
                    'gauge_first_index': i,
                    'gauge_second_index': j,
                    'gauge_distance': gauge_distance,
                    'river_distance': river_distance,
                    'point_distance': point_distance,
                    'provider_distance': provider_distance
                }

# Convert dictionary to DataFrame for visualization
dist_df = pd.DataFrame.from_dict(distances, orient='index')
dist_df.index.names = ['gauge_name1', 'gauge_name2']
dist_df.reset_index(inplace=True)

1000it [40:59,  2.46s/it]


In [13]:
dist_df

,gauge_name1,gauge_name2,gauge_first_index,gauge_second_index,gauge_distance,river_distance,point_distance,provider_distance
0,Bangs,Bangs,AT000001,CH000197,1.000000,1.000000,554.431769,False
1,Schruns (Vonbunweg),Schruns_(Vonbunweg),AT000002,CH000221,0.978947,1.000000,281.435851,False
2,Loruens-Aeule,Loruens-Aeule,AT000003,CH000215,1.000000,1.000000,356.336721,False
3,Kloesterle (OEBB),Kloesterle_(OEBB),AT000004,CH000227,0.976471,1.000000,179.092492,False
4,Buers (Bruecke L82),Buers_(Bruecke_L82),AT000005,CH000214,0.957895,1.000000,219.861947,False
5,Garsella,Garsella,AT000006,CH000218,1.000000,1.000000,484.251040,False
6,Beschling,Beschling,AT000007,CH000205,1.000000,1.000000,93.404431,False
7,Amerluegen,Amerluegen,AT000008,CH000201,1.000000,1.000000,548.837746,False
8,Gisingen,Gisingen,AT000009,CH000199,1.000000,1.000000,93.904040,False
9,Laterns,Laterns,AT000010,CH000209,1.000000,1.000000,351.603532,False


In [14]:
most_common_name = dist_df['gauge_first_index'].value_counts().idxmax()

print("The most common name in the column is:", most_common_name)

The most common name in the column is: AT000001


In [15]:
dist_df[dist_df.gauge_first_index == "FR000505"]

,gauge_name1,gauge_name2,gauge_first_index,gauge_second_index,gauge_distance,river_distance,point_distance,provider_distance


### Here we add the list of duplicated suspects

In [21]:
# First we create an empty column:
network_estreams["duplicated_suspect"] = np.nan

# Now we process both columns to ensure we cover all data:
# First column:
for gauge in tqdm.tqdm(dist_df.gauge_first_index):
    
    duplicated_list = str(dist_df.gauge_second_index[dist_df.gauge_first_index == gauge].tolist()).replace("[", "")
    duplicated_list = duplicated_list.replace("]", "")
    duplicated_list = duplicated_list.replace("'", "")
    network_estreams.loc[gauge, "duplicated_suspect"] = duplicated_list
    network_estreams.loc[gauge, "duplicated_suspect"] = network_estreams.loc[gauge, "duplicated_suspect"]

# Second column:
for gauge in tqdm.tqdm(dist_df.gauge_second_index):
    
    duplicated_list = str(dist_df.gauge_first_index[dist_df.gauge_second_index == gauge].tolist()).replace("[", "")
    duplicated_list = duplicated_list.replace("]", "")
    duplicated_list = duplicated_list.replace("'", "")
    network_estreams.loc[gauge, "duplicated_suspect"] = duplicated_list
    network_estreams.loc[gauge, "duplicated_suspect"] = network_estreams.loc[gauge, "duplicated_suspect"]

100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 7040.87it/s]


In [25]:
# Adjust the duplicated_suspect column to ensure we save the data as a list:
network_estreams['duplicated_suspect'] = network_estreams['duplicated_suspect'].str.replace(r'\s*,\s*', ',')
network_estreams['duplicated_suspect'] = network_estreams['duplicated_suspect'].str.split(',') 
network_estreams

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,area,...,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,geometry,duplicated_suspect
basin_id,,,,,,,,,,,,,,,,,,,,,
AT000001,200014,Bangs,AT,AT_EHYD,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,...,-0.440608,1996-01-01,2021-12-31,26,312,9497,0.0,9497,POINT (9.53484 47.27375),[CH000197]
AT000002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,102.0,...,-0.281373,1958-10-01,2021-12-31,64,759,23103,0.0,23103,POINT (9.91368 47.08030),[CH000221]
AT000003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,535.2,...,-0.205344,1985-01-02,2021-12-31,37,444,13513,0.0,13513,POINT (9.84777 47.13282),[CH000215]
AT000004,200592,Kloesterle (OEBB),AT,AT_EHYD,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,...,0.471471,1998-01-02,2021-12-31,24,288,8765,0.0,8765,POINT (10.06184 47.12899),[CH000227]
AT000005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,...,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957,POINT (9.80267 47.15077),[CH000214]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR0017,6682300,BASHTANOVKA,UA,UA_GRDC,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,...,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652,POINT (33.90000 44.68333),NaN
UAGR0018,6682500,YALTA,UA,UA_GRDC,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,...,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652,POINT (34.16667 44.50000),NaN
UAGR0019,6683010,PIONERSKOE,UA,UA_GRDC,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,...,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652,POINT (34.20000 44.88333),NaN


## Analysis of the duplicates:

In [ ]:
print("The number of duplicates suspects is", network_estreams.duplicated_suspect.count())

In [ ]:
network_duplicates = pd.DataFrame(network_estreams.groupby('gauge_country')['duplicated_suspect'].count())
network_duplicates.head(50)

## Save the data

In [ ]:
# Save the data:
network_estreams.to_csv("results/estreams_gauging_stations_duplicates.csv", encoding='utf-8')

# End